importing the libs

In [ ]:
import pickle
import numpy as np

In [ ]:
!pip install torch

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

Loading the training data

In [ ]:
with open('/home/jovyan/ChatBot/data/train.pk','rb') as f:
    train_data = pickle.load(f)

In [ ]:
train_data[0]

{'explicit_inform_slots': {'Cough': True,
  'Chest tightness and shortness of breath': False},
 'implicit_inform_slots': {'Pain behind the breastbone': False,
  'Chest tightness': False,
  'Hemoptysis': True,
  'Expectoration': True},
 'disease_tag': 'Esophagitis'}

Modeling the Data for some specific purpose

In [ ]:
all_diseases = []
all_symptoms = []

In [ ]:
with open("/home/jovyan/ChatBot/data/disease.txt") as file:
    for item in file:
        all_diseases.append(item.split("\n")[0])

In [ ]:
with open("/home/jovyan/ChatBot/data/symptom.txt") as file:
    for item in file:
        all_symptoms.append(item.split("\n")[0])

In [ ]:
def get_all_X_y(data):
    X = []
    y = []
    for conversation in data:
        sample = [0 for i in range(len(all_symptoms))]
        for symptom in list(conversation['implicit_inform_slots'].keys()):
            if(conversation['implicit_inform_slots'][symptom]==True):
                sample[all_symptoms.index(symptom)] = 1
        for symptom in list(conversation['explicit_inform_slots'].keys()):
            if(conversation['explicit_inform_slots'][symptom]==True):
                sample[all_symptoms.index(symptom)] = 1
        X.append(np.array(sample))
        y.append(all_diseases.index(conversation['disease_tag']))
    return torch.tensor(X, dtype=torch.float), torch.tensor(y, dtype=torch.float)

In [ ]:
def get_X_splitted(data):
    X_implicit = []
    explicit_questions = []
    for conversation in data:
        sample_implicit = [0 for i in range(len(all_symptoms))]
        sample_explicit = [0 for i in range(len(all_symptoms))]
        for symptom in list(conversation['implicit_inform_slots'].keys()):
            if(conversation['implicit_inform_slots'][symptom]==True):
                sample_implicit[all_symptoms.index(symptom)] = 1
        for symptom in list(conversation['explicit_inform_slots'].keys()):
            sample_explicit[all_symptoms.index(symptom)] = 1
        explicit_questions.append(np.array(sample_explicit))
        X_implicit.append(np.array(sample_implicit))
    return torch.tensor(X_implicit, dtype=torch.float), torch.tensor(explicit_questions, dtype=torch.float)

In [ ]:
def form_nn_model_classification(input_size, hidden_sizes, output_size):
    model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                          nn.ReLU(),
                          nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                          nn.ReLU(),
                          nn.Linear(hidden_sizes[1], output_size),
                          nn.Softmax(dim=1))
    return model

In [ ]:
def form_nn_model_questions(input_size, hidden_sizes, output_size):
    model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                          nn.ReLU(),
                          nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                          nn.SiLU(),
                          nn.Linear(hidden_sizes[1], hidden_sizes[2]),
                          nn.ReLU(),
                          nn.Linear(hidden_sizes[2], output_size),
                          nn.Sigmoid())
    return model

In [ ]:
class ModelMyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.len = len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
    def __len__(self):
        return self.len

In [ ]:
def makeDataLoader(X, y):
    data = ModelMyDataset(X, y)
    loader = DataLoader(dataset=data, batch_size=64, shuffle=True)
    return loader

In [ ]:
def train_model_classification(model, epochs, X, y, flag_4_questions):
    loss_function = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.003)
    loader = makeDataLoader(X, y)
    for epoch in range(epochs):
        running_loss = 0
        for i, data in enumerate(loader, 0):
            x_batch, y_batch = data
            optimizer.zero_grad()
            output = model(x_batch)
            if(flag_4_questions):
                loss = loss_function(output, y_batch)
            else:
                loss = loss_function(output, torch.tensor(y_batch, dtype=torch.long))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Training loss: {running_loss/len(loader)}")
    return model

In [ ]:
X_train, y_train = get_all_X_y(train_data)

/tmp/ipykernel_461/1753552437.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:204.)
  return torch.tensor(X, dtype=torch.float), torch.tensor(y, dtype=torch.float)


In [ ]:
X_train_implicit, explicit_questions_train = get_X_splitted(train_data)

In [ ]:
classification_model = form_nn_model_classification(118, [78, 60], 12)

In [ ]:
classification_model = train_model_classification(classification_model, 10, X_train, y_train, 0)

/tmp/ipykernel_461/2556389155.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss = loss_function(output, torch.tensor(y_batch, dtype=torch.long))


Training loss: 2.4849793116251626
Training loss: 2.4849728902180988
Training loss: 2.4849753697713215
Training loss: 2.4849756717681886
Training loss: 2.4849749326705934
Training loss: 2.4849682410558063
Training loss: 2.4849617083867392
Training loss: 2.4849597851435345
Training loss: 2.4849605321884156
Training loss: 2.484954086939494


In [ ]:
question_extraction_model = form_nn_model_questions(118, [78, 197, 78], 118)

In [ ]:
question_extraction_model = train_model_classification(question_extraction_model, 10, X_train_implicit, explicit_questions_train, 1)

Training loss: 13.940605926513673
Training loss: 13.93078244527181
Training loss: 13.928387546539307
Training loss: 13.92288153966268
Training loss: 13.932922395070394
Training loss: 13.923833115895588
Training loss: 13.909375317891438
Training loss: 13.924449729919434
Training loss: 13.918235778808594
Training loss: 13.92370309829712


**Making functions for the testing the model on some dataset**

Functions for testing the question generating accuracy

In [ ]:
def make_classification_predictions(model, X):
    prob_predictions = model(X[:])
    y_preds = []
    for probs in prob_predictions:
        y_preds.append(int(torch.argmax(probs)))
    return y_preds

In [ ]:
def make_questions_extraction_predictions(model, X):
    predictions = model(X[:])
    for i in range(len(predictions)):
        predictions[i] = torch.tensor(list(map(lambda x: 1 if x>0.5 else 0, predictions[i].tolist())))
    return predictions

In [ ]:
def calculate_accuracy(y_actual, y_pred):
    count = 0
    for i in range(len(y_actual)):
        if(int(y_actual[i]) == y_pred[i]):
            count += 1
    return count/len(y_actual)

In [ ]:
def calculate_accuracy_question_extraction(y_actual, y_pred):
    count = 0
    for i in range(len(y_actual)):
        if(y_actual[i] == y_pred[i]):
            count += 1
    return count/len(y_actual)

validation set

In [ ]:
with open('/home/jovyan/ChatBot/data/dev.pk','rb') as f:
    valid_data = pickle.load(f)

In [ ]:
X_valid, y_valid = get_all_X_y(valid_data)

In [ ]:
y_valid_predictions = make_classification_predictions(classification_model, X_valid)

In [ ]:
valid_accuracy = calculate_accuracy(y_valid, y_valid_predictions)
print(valid_accuracy)

0.06276150627615062


In [ ]:
X_valid_implicit, explicit_questions_valid = get_X_splitted(valid_data)

In [ ]:
explicit_questions_valid_predictions = make_questions_extraction_predictions(question_extraction_model, X_valid_implicit)

In [ ]:
valid_accuracy = calculate_accuracy_question_extraction(explicit_questions_valid.tolist(), explicit_questions_valid_predictions.tolist())
print(valid_accuracy)

0.0


Saving the Model

In [ ]:
torch.save(classification_model, '/home/jovyan/ChatBot/models/classification_model')

In [ ]:
torch.save(question_extraction_model, '/home/jovyan/ChatBot/models/question_extraction_model')

In [ ]:
lst = [0 for i in range(len(all_symptoms))]

In [ ]:
lst = torch.tensor(lst).type(torch.LongTensor)

In [ ]:
lst

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
question_extraction_model(torch.tensor(lst, dtype=torch.float))

tensor([0.5162, 0.5264, 0.4776, 0.5357, 0.5233, 0.5313, 0.4980, 0.4815, 0.4892,
        0.4972, 0.4930, 0.5290, 0.4832, 0.4860, 0.5409, 0.5211, 0.4715, 0.4815,
        0.5058, 0.5166, 0.4846, 0.5065, 0.4750, 0.4922, 0.4833, 0.5205, 0.4828,
        0.4900, 0.4991, 0.5294, 0.5197, 0.4864, 0.4927, 0.5279, 0.5271, 0.5063,
        0.4807, 0.5152, 0.4895, 0.4694, 0.5019, 0.4690, 0.4705, 0.4790, 0.5232,
        0.5002, 0.4970, 0.5010, 0.5255, 0.4821, 0.4705, 0.4842, 0.5224, 0.4895,
        0.5026, 0.4857, 0.4789, 0.4828, 0.4673, 0.5152, 0.4725, 0.4876, 0.4916,
        0.5032, 0.5062, 0.4980, 0.5244, 0.5099, 0.5113, 0.5085, 0.5086, 0.4773,
        0.5132, 0.4988, 0.4765, 0.4766, 0.5296, 0.4995, 0.4745, 0.5015, 0.5162,
        0.4959, 0.5320, 0.5034, 0.4889, 0.5133, 0.4755, 0.4921, 0.4967, 0.4928,
        0.4964, 0.5243, 0.4883, 0.5094, 0.4794, 0.4894, 0.4822, 0.5044, 0.5024,
        0.4729, 0.4816, 0.5109, 0.5175, 0.4899, 0.4700, 0.4930, 0.5080, 0.5013,
        0.5229, 0.5081, 0.4758, 0.4990, 

In [ ]:
import torch

In [ ]:
lst  = torch.tensor([0 for i in range(118)], dtype=torch.float)

In [ ]:
lst

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
question_extraction_model = torch.load('/home/jovyan/ChatBot/models/question_extraction_model')

In [ ]:
preds = question_extraction_model(lst[:])

In [ ]:
preds

tensor([0.5162, 0.5264, 0.4776, 0.5357, 0.5233, 0.5313, 0.4980, 0.4815, 0.4892,
        0.4972, 0.4930, 0.5290, 0.4832, 0.4860, 0.5409, 0.5211, 0.4715, 0.4815,
        0.5058, 0.5166, 0.4846, 0.5065, 0.4750, 0.4922, 0.4833, 0.5205, 0.4828,
        0.4900, 0.4991, 0.5294, 0.5197, 0.4864, 0.4927, 0.5279, 0.5271, 0.5063,
        0.4807, 0.5152, 0.4895, 0.4694, 0.5019, 0.4690, 0.4705, 0.4790, 0.5232,
        0.5002, 0.4970, 0.5010, 0.5255, 0.4821, 0.4705, 0.4842, 0.5224, 0.4895,
        0.5026, 0.4857, 0.4789, 0.4828, 0.4673, 0.5152, 0.4725, 0.4876, 0.4916,
        0.5032, 0.5062, 0.4980, 0.5244, 0.5099, 0.5113, 0.5085, 0.5086, 0.4773,
        0.5132, 0.4988, 0.4765, 0.4766, 0.5296, 0.4995, 0.4745, 0.5015, 0.5162,
        0.4959, 0.5320, 0.5034, 0.4889, 0.5133, 0.4755, 0.4921, 0.4967, 0.4928,
        0.4964, 0.5243, 0.4883, 0.5094, 0.4794, 0.4894, 0.4822, 0.5044, 0.5024,
        0.4729, 0.4816, 0.5109, 0.5175, 0.4899, 0.4700, 0.4930, 0.5080, 0.5013,
        0.5229, 0.5081, 0.4758, 0.4990, 